<a href="https://colab.research.google.com/github/karin-kaito/FWI_202506/blob/main/ensemble_csv_with_public_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
# kagglehub.login()


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

# waveform_inversion_path = kagglehub.competition_download('waveform-inversion')
# egortrushin_open_wfi_test_path = kagglehub.dataset_download('egortrushin/open-wfi-test')
# atom1231_caformer_28_2_path = kagglehub.dataset_download('atom1231/caformer-28-2')
atom1231_caformrt_28_8_path = kagglehub.dataset_download('atom1231/caformrt-28-8')
atom1231_convnext0small_33_2_path = kagglehub.dataset_download('atom1231/convnext0small-33-2')

print('Data source import complete.')


100%|██████████| 382M/382M [00:01<00:00, 218MB/s]

Extracting files...


100%|██████████| 475M/475M [00:03<00:00, 161MB/s]

Extracting files...


Data source import complete.


In [4]:
atom1231_caformrt_28_8_path

'/root/.cache/kagglehub/datasets/atom1231/caformrt-28-8/versions/1'

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 359.2 gigabytes of available RAM

You are using a high-RAM runtime!


## Find files to load and create Dataset

# 日本語へ変更済

In [6]:
import pandas as pd
import numpy as np
import os

def ensemble_submissions(file_paths, output_dir='.', weights=None):
    """
    複数の submission CSV ファイルを統合し、必要に応じて加重平均を計算します。

    Args:
        file_paths (list): すべての submission.csv ファイルのパスを含むリスト。
        output_dir (str): 結果ファイルが保存されるディレクトリ。
        weights (list, optional): file_paths に対応する重みのリスト。
                                  None の場合、単純平均を計算します。デフォルトは None です。
    """
    if not file_paths:
        print("エラー：ファイルパスリストが空です。")
        return

    # --- 新規追加：重み検証 ---
    if weights is not None:
        if len(weights) != len(file_paths):
            print(f"エラー：重みリストの長さ ({len(weights)}) がファイルリストの長さ ({len(file_paths)}) と一致しません。")
            return
        print(f"加重統合を開始します。使用する重みは: {weights}")
    else:
        print(f"単純平均統合を開始します（重みは提供されていません）。")

    # --- ステップ 1: すべての CSV ファイルを読み込み、'oid_ypos' をインデックスに設定します ---
    try:
        print("CSV ファイルを読み込んでいます...")
        dataframes = [pd.read_csv(f, index_col='oid_ypos') for f in file_paths]
    except FileNotFoundError as e:
        print(f"エラー：ファイルが見つかりません -> {e}。ファイルパスが正しいか確認してください。")
        return
    except KeyError:
        print("エラー：いずれかのファイルに 'oid_ypos' 列が見つかりません。すべての CSV にこの列があることを確認してください。")
        return

    # --- ステップ 2: データの位置合わせとスタック ---
    print("データを位置合わせしています...")
    master_index = dataframes[0].index
    master_columns = dataframes[0].columns
    aligned_data = [df.reindex(master_index).values for df in dataframes]
    data_array = np.stack(aligned_data, axis=0).transpose(1, 2, 0)
    print("データスタックが完了しました。配列の次元 (行数, 列数, ファイル数):", data_array.shape)

    # --- ステップ 3: 平均値 (加重または単純) と中央値を計算します ---

    # 平均値を計算
    if weights is not None:
        print("加重平均値を計算しています...")
        # np.average を使用して加重計算
        mean_data = np.average(data_array, axis=2, weights=weights)
        mean_output_filename = 'ensemble_weighted_mean.csv'
    else:
        print("単純平均値を計算しています...")
        mean_data = np.mean(data_array, axis=2)
        mean_output_filename = 'ensemble_mean.csv'

    # 計算中央值 (中央値に加重の概念はありません)
    print("中央値を計算しています...")
    median_data = np.median(data_array, axis=2)

    # --- ステップ 4: 結果 DataFrame を作成し、CSV ファイルとして保存します ---
    # 平均値の結果
    df_mean = pd.DataFrame(mean_data, index=master_index, columns=master_columns)
    mean_output_path = os.path.join(output_dir, 'submission.csv')
    df_mean.to_csv(mean_output_path)
    print(f"平均値統合結果が以下に保存されました: {mean_output_path}")

    # # 中央値の結果
    # df_median = pd.DataFrame(median_data, index=master_index, columns=master_columns)
    # median_output_path = os.path.join(output_dir, 'ensemble_median.csv')
    # df_median.to_csv(median_output_path)
    # print(f"中央値統合結果が以下に保存されました: {median_output_path}")

# 2. すべての submission.csv ファイルのパスリストを定義します。
#    以下のリストをあなたのファイルパスに置き換えてください。
submission_files = [
    "/root/.cache/kagglehub/datasets/atom1231/caformrt-28-8/versions/1/submission.csv",
    "/root/.cache/kagglehub/datasets/atom1231/convnext0small-33-2/versions/1/submission.csv",
  #"/kaggle/input/convnext-base-29-6/submission.csv" # このパスも同様に修正が必要です。
]

file_weights = [0.8, 0.2]
# 3. 統合関数を実行します。
ensemble_submissions(submission_files, weights=file_weights)
#単純平均
#ensemble_submissions(submission_files)

加重統合を開始します。使用する重みは: [0.8, 0.2]
CSV ファイルを読み込んでいます...
データを位置合わせしています...
データスタックが完了しました。配列の次元 (行数, 列数, ファイル数): (4607260, 35, 2)
加重平均値を計算しています...
中央値を計算しています...
平均値統合結果が以下に保存されました: ./submission.csv


In [7]:
!pwd

/content


In [8]:
!ls

sample_data  submission.csv


In [9]:
!ls -la

total 2318808
drwxr-xr-x 1 root root       4096 Jun  8 07:42 .
drwxr-xr-x 1 root root       4096 Jun  8 07:38 ..
drwxr-xr-x 4 root root       4096 Jun  5 13:29 .config
drwxr-xr-x 1 root root       4096 Jun  5 13:30 sample_data
-rw-r--r-- 1 root root 2374437553 Jun  8 07:45 submission.csv


In [7]:
import os

# Check the files in the specified directory
directory_path = '/kaggle/input/caformrt-28-8/'
if os.path.exists(directory_path):
    print(f"Files in {directory_path}:")
    for filename in os.listdir(directory_path):
        print(filename)
else:
    print(f"Directory not found: {directory_path}")

Directory not found: /kaggle/input/caformrt-28-8/


In [8]:
import os

# List all items in the /kaggle/input directory
input_directory = '/kaggle/input/'
if os.path.exists(input_directory):
    print(f"Contents of {input_directory}:")
    for item_name in os.listdir(input_directory):
        print(item_name)
else:
    print(f"Directory not found: {input_directory}")

Contents of /kaggle/input/:


In [9]:
import os

# List all items in the /kaggle/input directory
input_directory = '/kaggle/input/'
if os.path.exists(input_directory):
    print(f"Contents of {input_directory}:")
    for item_name in os.listdir(input_directory):
        print(item_name)
else:
    print(f"Directory not found: {input_directory}")

Contents of /kaggle/input/:


In [ ]:
# import pandas as pd
# import numpy as np
# import os

# def ensemble_submissions(file_paths, output_dir='.', weights=None):
#     """
#     將多個 submission CSV 檔案進行整合，可選擇計算加權平均。

#     Args:
#         file_paths (list): 一個包含所有 submission.csv 檔案路徑的列表。
#         output_dir (str): 輸出結果檔案的儲存目錄。
#         weights (list, optional): 一個與 file_paths 對應的權重列表。
#                                   若為 None，則計算簡單平均。預設為 None。
#     """
#     if not file_paths:
#         print("錯誤：檔案路徑列表是空的。")
#         return

#     # --- 新增：權重驗證 ---
#     if weights is not None:
#         if len(weights) != len(file_paths):
#             print(f"錯誤：權重列表的長度 ({len(weights)}) 與檔案列表的長度 ({len(file_paths)}) 不符。")
#             return
#         print(f"開始進行加權整合，使用的權重為: {weights}")
#     else:
#         print(f"開始進行簡單平均整合（未提供權重）。")

#     # --- 步驟 1: 讀取所有 CSV 檔案，並將 'oid_ypos' 設為索引 ---
#     try:
#         print("正在讀取 CSV 檔案...")
#         dataframes = [pd.read_csv(f, index_col='oid_ypos') for f in file_paths]
#     except FileNotFoundError as e:
#         print(f"錯誤：找不到檔案 -> {e}。請檢查您的檔案路徑是否正確。")
#         return
#     except KeyError:
#         print("錯誤：某個檔案中找不到 'oid_ypos' 欄位。請確保所有 CSV 都有此欄位。")
#         return

#     # --- 步驟 2: 資料對齊與堆疊 ---
#     print("正在對齊資料...")
#     master_index = dataframes[0].index
#     master_columns = dataframes[0].columns
#     aligned_data = [df.reindex(master_index).values for df in dataframes]
#     data_array = np.stack(aligned_data, axis=0).transpose(1, 2, 0)
#     print("資料堆疊完成。 陣列維度 (列數, 欄數, 檔案數):", data_array.shape)

#     # --- 步驟 3: 計算平均值 (加權或簡單) 與中位數 ---

#     # 計算平均值
#     if weights is not None:
#         print("正在計算加權平均值...")
#         # 使用 np.average 進行加權計算
#         mean_data = np.average(data_array, axis=2, weights=weights)
#         mean_output_filename = 'ensemble_weighted_mean.csv'
#     else:
#         print("正在計算簡單平均值...")
#         mean_data = np.mean(data_array, axis=2)
#         mean_output_filename = 'ensemble_mean.csv'

#     # 計算中位數 (中位數沒有加權的概念，維持原樣)
#     print("正在計算中位數...")
#     median_data = np.median(data_array, axis=2)

#     # --- 步驟 4: 建立結果 DataFrame 並儲存為 CSV 檔案 ---
#     # 平均值結果
#     df_mean = pd.DataFrame(mean_data, index=master_index, columns=master_columns)
#     mean_output_path = os.path.join(output_dir, 'submission.csv')
#     df_mean.to_csv(mean_output_path)
#     print(f"平均值整合結果已儲存至: {mean_output_path}")

#     # # 中位數結果
#     # df_median = pd.DataFrame(median_data, index=master_index, columns=master_columns)
#     # median_output_path = os.path.join(output_dir, 'ensemble_median.csv')
#     # df_median.to_csv(median_output_path)
#     # print(f"中位數整合結果已儲存至: {median_output_path}")

# # 2. 定義您所有 submission.csv 檔案的路徑列表。
# #    請將下面的列表換成您自己的檔案路徑。
# submission_files = [
#     "/kaggle/input/caformrt-28-8/submission.csv",
#     "/kaggle/input/convnext0small-33-2/submission.csv",
#   #"/kaggle/input/convnext-base-29-6/submission.csv"
# ]

# file_weights = [0.8, 0.2]
# # 3. 執行整合函式。
# ensemble_submissions(submission_files, weights=file_weights)
# #simple average
# #ensemble_submissions(submission_files)

In [11]:
import os

# Check the files in the specified directory
directory_path = '/kaggle/input/caformrt-28-8/'
if os.path.exists(directory_path):
    print(f"Files in {directory_path}:")
    for filename in os.listdir(directory_path):
        print(filename)
else:
    print(f"Directory not found: {directory_path}")

Directory not found: /kaggle/input/caformrt-28-8/
